# Linear solve

In this notebook we discuss how to linearly solve for the posterior over spherical harmonic coefficients of a map given a light curve. This is similar to what we did in the [Eclipsing Binary](EclipsingBinary_Linear.ipynb) notebook. The idea is to take advantage of the linearity of the `starry` solution to analytically compute the posterior over maps consistent with the data.

In [ ]:
%matplotlib inline

In [ ]:
%run notebook_setup.py

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import starry

np.random.seed(12)
starry.config.lazy = False
starry.config.quiet = True

We're going to demonstrate the linear solve feature using a map in reflected light, since the presence of a day/night terminator breaks many degeneracies and makes the mapping problem [much less ill-posed](NullSpace.ipynb). Let's begin by instantiating a reflected light map of the Earth. We'll give it the same obliquity as the Earth and observeit at an inclination of 60 degrees:

In [ ]:
map = starry.Map(ydeg=10, reflected=True)
map.obl = 23.5
map.inc = 60
map.load("earth")
map.show(projection="rect", illuminate=False)

Now we generate a dataset. We'll assume we have 10,000 observations over the course of a full orbit of the planet. We further take the planet's rotation period to be one-tenth of its orbital period. This will give us good coverage during all seasons, maximizing the amount of data we have for all the different regions of the planet:

In [ ]:
# Make the planet rotate 10 times over one full orbit
npts = 10000
nrot = 10
time = np.linspace(0, 1, npts)
theta = np.linspace(0, 360 * nrot, npts)

# Position of the star relative to the planet in the orbital plane
t = np.reshape(time, (1, -1))
p = np.vstack((np.cos(2 * np.pi * t), np.sin(2 * np.pi * t), 0 * t))

# Rotate to an observer inclination of 60 degrees
ci = np.cos(map.inc * np.pi / 180)
si = np.sin(map.inc * np.pi / 180)
R = np.array([[1, 0, 0], [0, ci, -si], [0, si, ci]])
xs, ys, zs = R.dot(p)

# Keywords to the `flux` method
kwargs = dict(theta=theta, xs=xs, ys=ys, zs=zs)

In [ ]:
# Compute the flux
flux0 = map.flux(**kwargs)
sigma = 0.005
flux = flux0 + sigma * np.random.randn(npts)

In [ ]:
fig, ax = plt.subplots(1, figsize=(12, 4))
ax.plot(time, flux)
ax.set_xlabel("Orbital phase", fontsize=18)
ax.set_ylabel("Normalized flux", fontsize=18);

Now the fun part. Let's instantiate a new map so we can do inference on this dataset:

In [ ]:
map = starry.Map(ydeg=10, reflected=True)
map.obl = 23.5
map.inc = 60

We now set the data vector (the flux and the covariance matrix) and the prior (`L` is the prior variance of the spherical harmonic coefficients, which we set to something small):

In [ ]:
map.set_data(flux, C=sigma ** 2)
map.set_prior(L=5e-4)

Finally, we call `solve`, passing in the `kwargs` from before. In this case, we're assuming we know the orbital information exactly. (When this is not the case, we need to do sampling for the orbital parameters; we cover this in more detail in the **Eclipsing Binary** tutorial).

In [ ]:
%%time
yhat, cho_ycov = map.solve(**kwargs)

The values returned are the posterior mean `yhat` of the spherical harmonic coefficients and the Cholesky factorization of the posterior covariance matrix `cho_ycov`. To view our map, we can set the map's coefficients equal to `yhat` (note that `yhat` does *not* include the zeroth coefficient, which is always fixed in `starry`):

In [ ]:
map[1:, :] = yhat

In [ ]:
map.show(projection="rect", illuminate=False)

We can also draw a random sample from the posterior (and automatically set the map coefficients) by calling

In [ ]:
map.draw()

In [ ]:
map.show(projection="rect", illuminate=False)

We can verify that we got a good fit to the data:

In [ ]:
fig, ax = plt.subplots(1, figsize=(12, 4))
ax.plot(time, flux)
plt.plot(time, map.flux(**kwargs))
ax.set_xlabel("Orbital phase", fontsize=18)
ax.set_ylabel("Normalized flux", fontsize=18);